## Install NewMethod

In [1]:
!git clone https://github.com/manhdo249/NewMethod.git

Cloning into 'NewMethod'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 11 (delta 0), reused 11 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (11/11), 8.27 KiB | 8.27 MiB/s, done.


In [2]:
! pip install topmost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 604.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.2.1
    Uninstalling sentence-transformers-3.2.1:
      Successfully uninstalled sentence-transformers-3.2.1


## Download a dataset

We download preprocessed dataset [NYT](https://github.com/BobXWu/TopMost/tree/main/data), news articles from New York Times.

In [3]:
import topmost
from topmost.data import download_dataset

download_dataset("NYT", cache_path="./datasets")
dataset = topmost.data.DynamicDataset("./datasets/NYT", read_labels = True, as_tensor=False)
docs = dataset.train_texts

100%|██████████| 15.1M/15.1M [00:00<00:00, 40.9MB/s]


train size:  8254
test size:  918
vocab size:  10000
average length: 175.429
num of each time slice:  11 [ 194  265  431  554  744  837  802  884 1283 1400  860]


## Train NewMethod

In [5]:
import sys
sys.path.append('/content/NewMethod')

from newmethod import NewMethod

model = NewMethod(num_topics=50, num_clusters = 10, epochs = 200, verbose=True)


2024-11-09 10:31:48,195 - NewMethod - use device: cuda


In [6]:
top_words, train_theta = model.fit_transform(docs)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

parsing texts: 100%|██████████| 8254/8254 [00:02<00:00, 3560.86it/s]
2024-11-09 10:32:12,182 - TopMost - Real vocab size: 10000
2024-11-09 10:32:12,241 - TopMost - Real training size: 8254 	 avg length: 175.429
Training NewMethod: 100%|██████████| 200/200 [01:56<00:00,  1.72it/s]

Topic 0: student chicago church campus abuse emanuel charleston lightfoot lori priests underage faculty superintendent alumni rabbi
Topic 1: banks investment firms funds shares finance estate cash investments wealth fund bitcoin investor trading hedge
Topic 2: refugees immigration border religious immigrants citizens migrants muslim jewish india orthodox refugee anti indian visas
Topic 3: brooklyn heroin pedestrians drinks bus youths subway cafe queens borough sidewalk singh nightclub bicycle bedroom
Topic 4: brexit netanyahu boris parliament prime minister britain spain london ireland referendum theresa british independence brussels
Topic 5: francis reservations feud pope ivanka vatican melania emeritus contenders rupture visions pivot thanked hails bully
Topic 6: income insurance education schools college communities teachers students families poor medicaid survey precincts census obamacare
Topic 7: iran ukraine ukrainian iraq military troops sanctions intelligence iraqi war forces a

# Eval

In [7]:
TD = topmost.evaluations.compute_topic_diversity(top_words)
TD

1.0

In [8]:
test_theta = model.transform(dataset.test_texts)
test_theta

array([[0.04025314, 0.01450145, 0.02574678, ..., 0.01493232, 0.01058759,
        0.02131775],
       [0.02409615, 0.01703929, 0.0126593 , ..., 0.01238607, 0.01915862,
        0.01789838],
       [0.02384714, 0.01725003, 0.01921969, ..., 0.00970077, 0.03015508,
        0.03009805],
       ...,
       [0.01067708, 0.05320613, 0.02206467, ..., 0.01098471, 0.02361837,
        0.01732743],
       [0.0170687 , 0.01259917, 0.01910839, ..., 0.02207149, 0.01203093,
        0.01553544],
       [0.01637117, 0.01500269, 0.01372831, ..., 0.01478111, 0.01646112,
        0.01466123]], dtype=float32)

In [9]:
# evaluate clustering
clustering_results = topmost.evaluations.evaluate_clustering(test_theta, dataset.test_labels)
print(clustering_results)
# evaluate classification
classification_results = topmost.evaluations.evaluate_classification(train_theta, test_theta, dataset.train_labels, dataset.test_labels)
print(classification_results)

{'Purity': 0.6612200435729847, 'NMI': 0.36852404017410173}
{'acc': 0.7745098039215687, 'macro-F1': 0.5998512995625003}


In [10]:
pwd

'/content'

In [11]:
!mkdir -p ./palmetto ./palmetto/wikipedia/wikipedia_bd
!wget --no-check-certificate -O ./palmetto/pametto.jar https://hobbitdata.informatik.uni-leipzig.de/homes/mroeder/palmetto/palmetto-0.1.0-jar-with-dependencies.jar
!wget --no-check-certificate -O wikipedia_corpus.zip https://hobbitdata.informatik.uni-leipzig.de/homes/mroeder/palmetto/Wikipedia_bd.zip
!unzip wikipedia_corpus.zip -d ./palmetto/wikipedia/


--2024-11-09 10:34:31--  https://hobbitdata.informatik.uni-leipzig.de/homes/mroeder/palmetto/palmetto-0.1.0-jar-with-dependencies.jar
Resolving hobbitdata.informatik.uni-leipzig.de (hobbitdata.informatik.uni-leipzig.de)... 139.18.4.2, 2001:638:902:2004::4:2
Connecting to hobbitdata.informatik.uni-leipzig.de (hobbitdata.informatik.uni-leipzig.de)|139.18.4.2|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files-redirect.dice-research.org/homes/mroeder/palmetto/palmetto-0.1.0-jar-with-dependencies.jar [following]
--2024-11-09 10:34:32--  https://files-redirect.dice-research.org/homes/mroeder/palmetto/palmetto-0.1.0-jar-with-dependencies.jar
Resolving files-redirect.dice-research.org (files-redirect.dice-research.org)... 131.234.26.214
Connecting to files-redirect.dice-research.org (files-redirect.dice-research.org)|131.234.26.214|:443... connected.
HTTP request sent, awaiting response... 302 Moved Tempora

In [12]:
with open("/content/palmetto/topics_NYT.txt", "w") as f:
    for topic in top_words:
        f.write(topic + "\n")

In [13]:
import subprocess
import pandas as pd

result = subprocess.run(
    ["java", "-jar", "/content/palmetto/pametto.jar", "/content/palmetto/wikipedia/wikipedia_bd", "C_V", "/content/palmetto/topics_NYT.txt"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

if result.returncode == 0:
    data = []

    for line in result.stdout.splitlines():
        line = line.strip()
        if line:
            parts = line.split("\t")
            if len(parts) >= 3:
                topic_id = parts[0].strip()
                coherence_score = float(parts[1].strip())
                words = parts[2].strip("[]").replace(",", "").split()

                data.append({
                    "Topic ID": topic_id,
                    "Coherence Score": coherence_score,
                    "Words": words
                })

    df = pd.DataFrame(data)
    print(df)

else:
    print("Error:", result.stderr)

average_coherence = df["Coherence Score"].mean()

print("Trung bình cộng của cột Coherence Score:", average_coherence)


   Topic ID  Coherence Score  \
0         0          0.42333   
1         1          0.43621   
2         2          0.36802   
3         3          0.35842   
4         4          0.36060   
5         5          0.44960   
6         6          0.41024   
7         7          0.36075   
8         8          0.52661   
9         9          0.48148   
10       10          0.42327   
11       11          0.41974   
12       12          0.40988   
13       13          0.42929   
14       14          0.50881   
15       15          0.32989   
16       16          0.39788   
17       17          0.30890   
18       18          0.57174   
19       19          0.42023   
20       20          0.43347   
21       21          0.44156   
22       22          0.46222   
23       23          0.31266   
24       24          0.43346   
25       25          0.49844   
26       26          0.39573   
27       27          0.41672   
28       28          0.36568   
29       29          0.39241   
30      

## Topic info

We can get the top words and their probabilities of a topic.

In [14]:
model.get_topic(topic_idx=36)

(('electric', 0.0072164773),
 ('tesla', 0.0069298004),
 ('cars', 0.0067343432),
 ('vehicles', 0.0066798),
 ('olympic', 0.0055477703))

## Visualize topic-word distributions

In [15]:
fig = model.visualize_topic(top_n=5)
fig.show()

## Visualize topic hierarchy

We use the learned topic embeddings and `scipy.cluster.hierarchy` to build a hierarchy of discovered topics.

In [16]:
fig = model.visualize_topic_hierarchy()
fig.show()

## Visualize topic weights

We plot the weights of topics in the given dataset.

In [17]:
fig = model.visualize_topic_weights(top_n=20, height=500)
fig.show()

## Get topic activity over time


Topic activity refers to the weight of a topic at a time slice.
We additionally input the time slices of documents, `time_slices` to compute and plot topic activity over time.


In [18]:
time_slices = dataset.train_times
act = model.topic_activity_over_time(time_slices)
fig = model.visualize_topic_activity(top_n=6, topic_activity=act, time_slices=time_slices)
fig.show()